### Red neuronal recurrente 
codigo original [aqui](https://gist.github.com/karpathy/d4dee566867f8291f086)

Importando librerias necesarias

In [1]:
import numpy as np

### Cargando datos

In [2]:
data = open('municipios.txt', encoding="utf8").read() #Carcgando archivo txt con nombres de municipios mexicanos
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('El archivo tiene %d caracteres, %d sin repetir.' % (data_size, vocab_size)) 
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

El archivo tiene 35530 caracteres, 68 sin repetir.


### Hiperparametros

In [3]:
hidden_size = 100 # numero de capas ocultas 
seq_length = 25 # numero de pasos para desenrollar los parametros
learning_rate = 1e-1

### Parametros del modelo

In [4]:
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

### Funcion de perdida 

In [5]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

### Funcion muestra

In [6]:
def sample(h, seed_ix, n):
  """
  sample a sequence of integers from the model
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

### Generando el texto

In [ ]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress

  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter

----
 na Necaulitla
Santiago Telial
Santas
San Aniles
San Satragdel Caresdoa
Santiago Felapa Pistlel
Santiago Techapan
Santis des Huzacuitlán y Pancios Juem
Sano Cantóbán
Santa María Yeatotetepec
Santiago T 
----
iter 0, loss: 105.437810
----
 zrrrhBrazrBandBaBBrrBanBlánBpivaeaatz
BaBB
Ba
B
BBlvrrBruB
Bon
B
B
Brul BaB
BrnaBaBrBá
BaBBalalz
Bná
B
B
B Balan
BniBBailrBadadag
B
Bail
BaaBcB
BBa
BBlBaán Bac
B
Brshuslanáauc
Bn
BrnhBácocn
Bla
Bc
Bre 
----
iter 100, loss: 105.237867
----
 Ceniso
Áácua yahoaleclanato
C
Cnal de
Ccálioah
CJlicen dea Hotcalepao
Cohnáhilanaro
Cla
Cituao
Casisonanuyanianahoa dCol dntz denanolancenoáli Giditan deáyupanancano
Coccicol eosholh
Blát Clnuetoaicl  
----
iter 200, loss: 101.890315
----
 gíniera
eres
Gn Fl Génale Le
Grolalinq
C
Menalo
rerenaen
Grargenerenananarín
Col
Canntlan iconíno
Cero
Gonorereneela
Ceq
Hino
Hanan Zarerenternnerevelomoreonolerorananjorderenemareizhellengere
C Greal 
----
iter 300, loss: 98.576999
----
 
Itral
Ilálajálamalaanmíz
Ilazualala


----
 ajign
Santepan
Sacés Suina Ella Sin Cumuel Aras Culinta
Sayo
San Olángus
San Larongifuía Avo Acosen de Juca
San
Saméyaxe
Sarharlascilteajunalicatsénanixtan ni Sicíián
Sal dasulnga Carnan San Amuemualt 
----
iter 3500, loss: 54.298729
----
 na
San Jencos
San Jo Jerotlan
Sanarimimícómómaxtotala
San Jusro
Sanc San Jearalóla
San
Sat Juón
Kanto
San Joa
San Jocpe drotlalo
San Janco
Jan Jocpaepen
Sanvilocón
San Os
Sac Serpan
San
Huanac
Sano Sa 
----
iter 3600, loss: 53.632690
----
  Lapoz Tlo
San Man Teteto To Maán
Sal Tucte Mepen
San Mono
San Mapen C Catlatlitlin Marnco Can Cuo
San Metonutín Manitepe San Maa
San Man
San Mapateteto de Yatlthan
San Matlteato
San Matquoo
San Ravoo 
----
iter 3700, loss: 52.481102
----
 a
Sanes
San Sai
Sa
San Sanbpan
Ququeltia
San Salal
San Sal -ujiaya
San Side Saa
Sapa deblepec
San Sa derere Sebafilal de Sebte Selsol
San Sanucbi dehuxtatocpec Saro
San ie Tejejos Aypea
San Sacuiatepe 
----
iter 3800, loss: 51.253281
----
  Heda María Ora
Santajatla dua
San

----
 án
Tulá
Vezhix
Tuakiten
Tuas
Tuquilco
Tialá Trído de de Tango Tardeba
Tguxtisuci Fla
Tepecha de Tetán demquil
Tuxuxocelu
Tonyuducul
Tupumun
Tucuxtixc
Tuapa
Teybltiás de Junxcanc
Tentlás Jús
Tuxpa
Zotz 
----
iter 7000, loss: 46.116848
----
 arueco
Zuepehtakíhireniteltepacpiacán
Zichiguitepoten
Zaco
Zaroielláhivuititico
Ziéca
Teconciriálliutatihuecongo
Zaganuatlán
Zilipa
Ailán
Zidraguicihuece Coultapila
Tunito Suajeran Indaade
Yepacuean
Z 
----
iter 7100, loss: 46.407668
----
 o
Ayuntomolco
Aycapo
Atuuntlán
Atzo Aludatoyas Altedris
Aztoyoyui
A
Ayutlo Aldo
Atzos Áxuimroz de Alticocaviela
Alltón
Apaz
Ayalaltenar Grexoyote
Ciguato
Asotla de Nantetla
Atonucacogdralón
Axitz
Ayot 
----
iter 7200, loss: 46.849982
----
 izan
Zara
Mentetía
Caz
Cayatá Cantlán
Quitla
Niteruaz Caxtes
Caraxo
Cehuaca Garitla Catlán Jal Colecaz
Cholapan Nengo Cárohútlasta de Jucatla Áoro
Cuipen
Cáres Ristia
C.icelás
Cayá
Coitla
Calizotilé d 
----
iter 7300, loss: 47.283710
----
 mjoyav
Zacomonimo Isapec
uronagro 

----
 s
Noconzo Narmez Nizatzan Naón Comalo
Nillán de Huacas
Niquicillántlán
Nimanchan Nacoa
Natan Apac
Netepal
Ola Nico
Moruevis Naglos Niio Coraleuitín del Pousco
Nióno Cilxco
Natlán Nazalán
Norrol ándalo 
----
iter 10500, loss: 46.517549
----
 stipal Asagageno
Díoga
Saltento de Astingo Avas Atugulb Nimana Arín
San de Cogránechin
Anguguivasistingo
San Tinitanoas
San Aguibapan
Són
Ltlán de Ayojac
San Zava
Sda
Soliín
San Antaranguc
San Astilde 
----
iter 10600, loss: 47.034467
----
 ncena
San Jorras Terrego
Imez Jolostoro
San Jachión
San Jobiacamo Cosghilco
Sen Astomisto Safente Anda Silánés
San Jelego
Sen Fa Cafingo
Santo Siltoci Piepac Oreyac Sanatla Oscaliszo Díazamisco
San Tr 
----
iter 10700, loss: 46.356089
----
 
Sán Juzinto Vilco Mormío Tiltlatíatos
San Matínel
San Temaxtlán
San Zitepen Taltla Yolta
San Losolco
Sansri
San Mortín Martío greríaric
Soro
San Jalvecel
San Maytos As R. Maranteápac
San tío As de Mo 
----
iter 10800, loss: 45.326635
----
 a
San Payoteo Huixtlpe Guaulde

----
 ulapa
Santiago Santla
Santiago Montixta Tehuanila
Santiago Nenlixga
Santiago Sannién Roracalapulán
Santiago Miguicpec
Santiago Santiago Tlahuicúta
Santiago del Taltepec
Santiago del Rojumutepac
Santia 
----
iter 13900, loss: 41.441893
----
 tepecamom
Tenelecencoz
Teata
Teo
Sanuma
Tezéteutlán
Tamumting
Tenam
Trotomán
Tacán
Teva
Tehuani
Temine de Telamín Tolihuelco
Tenaa
Teblepen
Tejuta Gueca
Temzapaga
Targoíte
Havecán
Rón
Tláno
Tebhuarca  
----
iter 14000, loss: 41.649590
----
 enjoguela
Tlatipuca
Tloncú
Tuango
Tlancono Tepa
Canz
Tirna
Tlango
Tetaya
Temipa Tengeno
Tehuco
Totla Tramtopeb Vi de de Gerutechipolaralcán de Zalepac
Tlatermelo
Tejuixco de de Zlo
Toteco
Tolo
Tlán
Tl 
----
iter 14100, loss: 41.815554
----
 acac
Zotpacanten
-
Zanmic
de merik Otinutlá
Zacodeza
Zatlán
Zarola
Zamianoloucón de Gorarec
Zahitlepo
Tesitlán
Zapelto
Zopoctistrelpaz
Zimiz
Tiltom de Yonatpan de Zacheriatla
Zotezoyan Huetlano
Zarotz 
----
iter 14200, loss: 42.106389
----
 yanad
Axhinco
Atzatín de Anuza

----
 auco
Cuecolchipdumiatoyapova
Cilzante
Caáres
Cueperris
Comalpamtepezuc
Crupac
Cuautzac
Cuius
Guiltes
Chuxagpe
Atato
Cuanquilo
Cuertaré de Cinoquiárulez
Cumúc del Mavuché de Guilllexta
Cuilcaz del Cosi 
----
iter 17300, loss: 42.604864
----
 oraguipan de Hoita Huaro
Huadadar
Huiula de Holipan
Hora
Hilana
Bacrartuapapumuma Huadán
Geradaldas
Sautamalgo
Hüicheo
Huepeo
Gosstepedo
Huijalón
HaHualpa
Gura Hinutes
Oziño Higutuc Picano
Hiañucralos 
----
iter 17400, loss: 43.570212
----
 
La Lantes
San Maano
La Potirdea
La Liro
La Lar
Linila
La Lecotzaz La La Orúaz
Lo de Fran Manga
Camáride
Ferre
Lititzarión Lucad
La del Restza
Sana Lalán
Reyal
Vicitindo Coyiqueé Ntitlán de Fraa
La Ma 
----
iter 17500, loss: 43.762291
----
 
Neranio
Fzaquisara
Nacatla
Juaquerianaro
Morioco
Legonaz
Narña
Nenujoc
Nanas
Nepan
Noauzeta Naralaán
Nuisto Nadala
Mosorita de Verulán
Nenda
Moraaca
Naraean
Naraz
Napo
Dorio
Nazala ixtes de Martaquía 
----
iter 17600, loss: 44.107317
----
 Galla de Ría de Custajo
San Ot

----
 pa
Nuattlaca
Juantlapan
San Juan Hilco
San Jeafac
San Juan Indratla
San Joscas
San Joxto -2 Juan
San Juancistlamo
San Juan Dilamica
San Juen Zastan Ciovo
San Juan del Rampan
San Los Chitlango de Sutla 
----
iter 20700, loss: 42.798811
----
 la
Sen la Ses Elbllo Atlao Atláo de Bosotlán
Santiáhuacoos
San Miguel Rolmorham
Santos
San Pedalola -cotzoño Los
San Padros
San Pebanuxtpan
San Nicatiala
San Mallehueip
San Pedro de Sihuayan Miguel Zo 
----
iter 20800, loss: 41.779481
----
 eacatian Chitiaca
Sinoa Coyaro
Sana Jutrre
Santiaga
Santa Cotumuitlanen Tiguachtánaz
Santo Cuitango
Santa Critec
Sanca de Sucima
Santa Chuagunar
Santa del La Campar
Santa Chula Niixtepec Yaba
Santa Cr 
----
iter 20900, loss: 41.049522
----
 
Santiago Tllenazope
Santiago elalepeguinas
Sayte Nucitla
Santiago Tila
Santiago Lacotlán
Santiago Santo Marina
Santiago M. Moxtopec
Santiago Río Axtitlán
Santiago
Santiago
Ypesvigo
Santiago Jositepey 
----
iter 21000, loss: 39.709628
----
 eacaimo
Toyahualepac
Tacacalá
